In [1]:
from skyfield.api import Topos, load

stations_url = 'http://celestrak.com/NORAD/elements/stations.txt'
satellites = load.tle_file(stations_url)
print('Loaded', len(satellites), 'satellites')
#for sat in satellites:
#    print(sat)

Loaded 70 satellites


In [2]:
by_name = {sat.name: sat for sat in satellites}
satellite = by_name['ISS (ZARYA)']
print(satellite)

EarthSatellite 'ISS (ZARYA)' number=25544 epoch=2020-06-18T17:13:55Z


In [3]:
ts = load.timescale(builtin=True)
t_now = ts.now()
dt_now = ts.now().utc_datetime()
dt_morning = dt_now.replace(hour=0,minute=0,second=0, microsecond=0)
dt_evening = dt_now.replace(hour=23,minute=59,second=59, microsecond=0)

print(dt_morning.strftime('%Y-%m-%d %H:%M'))
print(dt_evening.strftime('%Y-%m-%d %H:%M'))

['__add__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__rsub__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', 'astimezone', 'combine', 'ctime', 'date', 'day', 'dst', 'fold', 'fromisocalendar', 'fromisoformat', 'fromordinal', 'fromtimestamp', 'hour', 'isocalendar', 'isoformat', 'isoweekday', 'max', 'microsecond', 'min', 'minute', 'month', 'now', 'replace', 'resolution', 'second', 'strftime', 'strptime', 'time', 'timestamp', 'timetuple', 'timetz', 'today', 'toordinal', 'tzinfo', 'tzname', 'utcfromtimestamp', 'utcnow', 'utcoffset', 'utctimetuple', 'weekday', 'year']
2020-06-19 00:00
2020-06-19 23:59


In [4]:
from skyfield.api import EarthSatellite

ts = load.timescale(builtin=True)
line1 = '1 25544U 98067A   14020.93268519  .00009878  00000-0  18200-3 0  5082'
line2 = '2 25544  51.6498 109.4756 0003572  55.9686 274.8005 15.49815350868473'
satellite_tle = EarthSatellite(line1, line2, 'ISS (ZARYA)', ts)
print(satellite_tle)
print(satellite_tle.epoch.utc_jpl())

EarthSatellite 'ISS (ZARYA)' number=25544 epoch=2014-01-20T22:23:04Z
A.D. 2014-Jan-20 22:23:04.0004 UT


In [5]:
t = ts.utc(2014, 1, 23, 11, 18, 7)

days = t - satellite.epoch
print('{:.3f} days away from epoch'.format(days))

if abs(days) > 14:
    satellites = load.tle_file(stations_url, reload=True)

-2338.247 days away from epoch


[#################################] 100% stations.txt


In [6]:
bluffton = Topos('40.8939 N', '83.8917 W')
t0 = ts.utc(2014, 1, 23)
t1 = ts.utc(2014, 1, 24)

tm = ts.utc(dt_morning)
te = ts.utc(dt_evening)
t, events = satellite.find_events(bluffton, tm, te, altitude_degrees=30.0)
for ti, event in zip(t, events):
    name = ('rise above 30°', 'culminate', 'set below 30°')[event]
    print(ti.utc_strftime('%Y %b %d %H:%M:%S'), name)

2020 Jun 19 13:04:16 rise above 30°
2020 Jun 19 13:05:47 culminate
2020 Jun 19 13:07:18 set below 30°
2020 Jun 19 19:34:02 rise above 30°
2020 Jun 19 19:35:22 culminate
2020 Jun 19 19:36:42 set below 30°


In [7]:
# You can instead use ts.now() for the current time
# t = ts.utc(2014, 1, 23, 11, 18, 7)
t = ts.now()
geocentric = satellite.at(t)
print(geocentric.position.km)

[ 6318.59033869 -2496.35142586   171.68301762]


In [8]:
subpoint = geocentric.subpoint()
print('Latitude:', subpoint.latitude)
print('Longitude:', subpoint.longitude)
print('Elevation (m):', int(subpoint.elevation.m))

Latitude: 01deg 33' 42.0"
Longitude: -75deg 50' 30.4"
Elevation (m): 417894


In [9]:
difference = satellite - bluffton
print(difference)

Sum of 2 vectors:
 - Topos 40deg 53' 38.0" N -83deg 53' 30.1" E
 + EarthSatellite 'ISS (ZARYA)' number=25544 epoch=2020-06-18T17:13:55Z


In [10]:
topocentric = difference.at(t)
print(topocentric.position.km)

alt, az, distance = topocentric.altaz()

if alt.degrees > 0:
    print('The ISS is above the horizon')

print(alt)
print(az)
print(int(distance.km), 'km')

[ 2112.66111059  -110.75908897 -3973.61149026]
-14deg 58' 53.0"
167deg 21' 48.6"
4501 km


In [11]:
ra, dec, distance = topocentric.radec()  # ICRF ("J2000")

print(ra)
print(dec)

23h 47m 59.75s
-61deg 58' 08.5"


In [12]:
ra, dec, distance = topocentric.radec(epoch='date')

print(ra)
print(dec)

23h 49m 04.08s
-61deg 51' 25.8"


In [13]:
eph = load('de421.bsp')
satellite = by_name['ISS (ZARYA)']

two_hours = ts.utc(2014, 1, 20, 0, range(0, 120, 20))
sunlit = satellite.at(two_hours).is_sunlit(eph)
print(sunlit)

[ True False  True  True  True False]


In [14]:
for ti, sunlit_i in zip(two_hours, sunlit):
    print('{}  {} is in {}'.format(
        ti.utc_strftime('%Y-%m-%d %H:%M'),
        satellite.name,
        'sunlight' if sunlit_i else 'shadow',
    ))

2014-01-20 00:00  ISS (ZARYA) is in sunlight
2014-01-20 00:20  ISS (ZARYA) is in shadow
2014-01-20 00:40  ISS (ZARYA) is in sunlight
2014-01-20 01:00  ISS (ZARYA) is in sunlight
2014-01-20 01:20  ISS (ZARYA) is in sunlight
2014-01-20 01:40  ISS (ZARYA) is in shadow
